In [17]:
import numpy as np
import pandas as pd

In [18]:
key_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date']

In [3]:
TEST_SIZE = 28

df_sales = pd.read_csv('./data/sales_train_evaluation.csv')
df_calendar = pd.read_csv('./data/calendar.csv', parse_dates=['date'])
df_prices = pd.read_csv('./data/sell_prices.csv')

# Add null sales for the remaining days 1942-1969
# d_1 to d_1941: train set & valid set
# d_1942 - d_1969: test set (forecast F1 to F28 in sample submission)

series_d = pd.Series(df_sales.columns)
series_d = series_d[series_d.str.contains('d_')].reset_index(drop=True)
series_d = series_d.apply(lambda x: x.split('_')[1]).astype(int)
max_d = max(series_d.values)

for d in range(max_d+1, max_d+1+TEST_SIZE):
    col = 'd_' + str(d)
    df_sales[col] = np.nan
    

# Trimming

In [4]:
# lag and ma params
LAG = [28]      # due to TEST_SIZE = 28, LAG at least 28
MA = [7, 28]#, 180, 360]

In [5]:
START_DATE = '2016'

start_date = pd.to_datetime(START_DATE) - pd.Timedelta(max(LAG)+max(MA),'days')
df_calendar_trim = df_calendar[(df_calendar.date>=start_date)].copy()

# For trimming
d_min = int(df_calendar_trim.d.min().split('_')[1])
d_max = int(df_calendar_trim.d.max().split('_')[1])
week_min = df_calendar_trim.wm_yr_wk.min()
week_max = df_calendar_trim.wm_yr_wk.max()

# preprocess on calendar
df_calendar_trim['is_holiday'] = df_calendar_trim['event_name_1'].notnull() | df_calendar_trim['event_name_2'].notnull()
df_calendar_trim['is_weekend'] = df_calendar_trim.weekday.isin(['Saturday','Sunday'])
df_calendar_trim = df_calendar_trim[['date', 'wm_yr_wk', 'weekday', 'd', 'is_holiday', 'is_weekend']]

df_calendar_trim

,date,wm_yr_wk,weekday,d,is_holiday,is_weekend
1742,2015-11-06,11540,Friday,d_1743,False,False
1743,2015-11-07,11541,Saturday,d_1744,False,True
1744,2015-11-08,11541,Sunday,d_1745,False,True
1745,2015-11-09,11541,Monday,d_1746,False,False
1746,2015-11-10,11541,Tuesday,d_1747,False,False
...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,d_1965,False,False
1965,2016-06-16,11620,Thursday,d_1966,False,False
1966,2016-06-17,11620,Friday,d_1967,False,False
1967,2016-06-18,11621,Saturday,d_1968,False,True


In [6]:
df_sales_trim = df_sales[[col for col in key_cols if col != 'date'] + [f"d_{n}" for n in range(d_min, d_max+1)]]
df_sales_trim

,id,item_id,dept_id,cat_id,store_id,state_id,d_1743,d_1744,d_1745,d_1746,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,3,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,3,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,3,0,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_prices_trim = df_prices[(df_prices.wm_yr_wk>=week_min) & (df_prices.wm_yr_wk<=week_max)]
df_prices_trim

,store_id,item_id,wm_yr_wk,sell_price
120,CA_1,HOBBIES_1_001,11540,8.26
121,CA_1,HOBBIES_1_001,11541,8.26
122,CA_1,HOBBIES_1_001,11542,8.26
123,CA_1,HOBBIES_1_001,11543,8.26
124,CA_1,HOBBIES_1_001,11544,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [8]:
df = pd.melt(df_sales_trim, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')
df = pd.merge(df, df_calendar_trim, how='left', on='d')
df = pd.merge(df, df_prices_trim, how='left', on=['store_id','item_id','wm_yr_wk'])
df['d'] = df['d'].apply(lambda x: x.split('_')[1]).astype(int)
df


,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,weekday,is_holiday,is_weekend,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1743,0.0,2015-11-06,11540,Friday,False,False,8.26
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1743,0.0,2015-11-06,11540,Friday,False,False,3.97
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1743,0.0,2015-11-06,11540,Friday,False,False,2.97
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1743,1.0,2015-11-06,11540,Friday,False,False,4.64
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1743,2.0,2015-11-06,11540,Friday,False,False,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6921225,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,Sunday,True,True,2.98
6921226,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,Sunday,True,True,2.48
6921227,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,Sunday,True,True,3.98
6921228,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,Sunday,True,True,1.28


In [9]:
# utils
def create_lag_ma(df, LAG:list, MA:list, key:list):
    key_name = '_'.join(key)+'_' if len(key) > 0 else ''
    for lag in LAG:
        df_temp = df[key + ["sold", "date"]].groupby(key+["date"]).sum().shift(lag).reset_index().rename(columns={'sold':f"{key_name}sold_lag{lag}"})

    for ma in MA :
        for lag in LAG:
            if len(key)==0:
                df_temp[f"{key_name}sold_lag{lag}_ma{ma}"] = df_temp[key + [f"{key_name}sold_lag{lag}"]].transform(lambda x : x.rolling(ma).mean())
            else:
                df_temp[f"{key_name}sold_lag{lag}_ma{ma}"] = df_temp[key + [f"{key_name}sold_lag{lag}"]].groupby(key)[f"{key_name}sold_lag{lag}"].transform(lambda x : x.rolling(ma).mean())

    df = df.merge(df_temp, how='left', on=key+["date"])
    return df

In [10]:
# lag and ma
df = create_lag_ma(df, LAG, MA, key=['id'])                     # id lv
df = create_lag_ma(df, LAG, MA, key=['item_id'])                # item lv
df = create_lag_ma(df, LAG, MA, key=['cat_id'])                 # cat lv
df = create_lag_ma(df, LAG, MA, key=['store_id'])               # store lv
df = create_lag_ma(df, LAG, MA, key=['dept_id','store_id'])     # dept-store lv
df = create_lag_ma(df, LAG, MA, key=['cat_id','state_id'])      # cat-state lv
df = create_lag_ma(df, LAG, MA, key=[])                         # global lv (like prospect index)

df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,store_id_sold_lag28_ma28,dept_id_store_id_sold_lag28,dept_id_store_id_sold_lag28_ma7,dept_id_store_id_sold_lag28_ma28,cat_id_state_id_sold_lag28,cat_id_state_id_sold_lag28_ma7,cat_id_state_id_sold_lag28_ma28,sold_lag28,sold_lag28_ma7,sold_lag28_ma28
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1743,0.0,2015-11-06,11540,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1743,0.0,2015-11-06,11540,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1743,0.0,2015-11-06,11540,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1743,1.0,2015-11-06,11540,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1743,2.0,2015-11-06,11540,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6921225,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,...,4099.107143,2632.0,1968.285714,2079.642857,10497.0,8991.714286,9901.5,54338.0,43372.285714,43991.571429
6921226,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,...,4099.107143,2632.0,1968.285714,2079.642857,10497.0,8991.714286,9901.5,54338.0,43372.285714,43991.571429
6921227,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,...,4099.107143,2632.0,1968.285714,2079.642857,10497.0,8991.714286,9901.5,54338.0,43372.285714,43991.571429
6921228,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1969,NaN,2016-06-19,11621,...,4099.107143,2632.0,1968.285714,2079.642857,10497.0,8991.714286,9901.5,54338.0,43372.285714,43991.571429


In [11]:
# trimming on start & end date (due to lag & ma)
df = df[(df.date>=START_DATE)].copy()

df = df.sort_values(['id', 'date']).reset_index(drop=True)
df.isnull().sum()

id                                       0
item_id                                  0
dept_id                                  0
cat_id                                   0
store_id                                 0
state_id                                 0
d                                        0
sold                                853720
date                                     0
wm_yr_wk                                 0
weekday                                  0
is_holiday                               0
is_weekend                               0
sell_price                             513
id_sold_lag28                            0
id_sold_lag28_ma7                        0
id_sold_lag28_ma28                       0
item_id_sold_lag28                       0
item_id_sold_lag28_ma7                   0
item_id_sold_lag28_ma28                  0
cat_id_sold_lag28                        0
cat_id_sold_lag28_ma7                    0
cat_id_sold_lag28_ma28                   0
store_id_so

In [12]:
# trimming on sell price
df = df[df.sell_price.notnull()].reset_index(drop=True)
df.isnull().sum()

id                                       0
item_id                                  0
dept_id                                  0
cat_id                                   0
store_id                                 0
state_id                                 0
d                                        0
sold                                853720
date                                     0
wm_yr_wk                                 0
weekday                                  0
is_holiday                               0
is_weekend                               0
sell_price                               0
id_sold_lag28                            0
id_sold_lag28_ma7                        0
id_sold_lag28_ma28                       0
item_id_sold_lag28                       0
item_id_sold_lag28_ma7                   0
item_id_sold_lag28_ma28                  0
cat_id_sold_lag28                        0
cat_id_sold_lag28_ma7                    0
cat_id_sold_lag28_ma28                   0
store_id_so

In [13]:
df = df.sort_values(['id', 'date']).reset_index(drop=True)
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,store_id_sold_lag28_ma28,dept_id_store_id_sold_lag28,dept_id_store_id_sold_lag28_ma7,dept_id_store_id_sold_lag28_ma28,cat_id_state_id_sold_lag28,cat_id_state_id_sold_lag28_ma7,cat_id_state_id_sold_lag28_ma28,sold_lag28,sold_lag28_ma7,sold_lag28_ma28
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1799,0.0,2016-01-01,11548,...,4069.071429,253.0,239.000000,247.500000,9773.0,9218.571429,9882.571429,38187.0,35539.428571,36817.714286
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1800,0.0,2016-01-02,11549,...,4064.500000,367.0,257.857143,248.035714,12172.0,9700.142857,9858.428571,47025.0,37369.857143,36806.428571
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1801,0.0,2016-01-03,11549,...,4055.678571,341.0,280.000000,250.357143,13253.0,10092.142857,9833.321429,48415.0,38888.857143,36806.571429
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1802,1.0,2016-01-04,11549,...,4062.535714,338.0,300.142857,255.500000,9725.0,10210.428571,9827.357143,35408.0,39373.428571,36752.750000
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1803,0.0,2016-01-05,11549,...,4049.392857,299.0,305.142857,258.964286,8852.0,10209.714286,9800.535714,32913.0,39101.285714,36736.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5213272,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,2016-06-15,11620,...,4072.857143,133.0,155.285714,159.357143,2236.0,2641.000000,2697.678571,37096.0,45291.428571,43596.428571
5213273,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,2016-06-16,11620,...,4079.000000,141.0,158.142857,159.321429,2590.0,2683.714286,2704.785714,36963.0,44785.714286,43665.357143
5213274,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,2016-06-17,11620,...,4089.642857,230.0,167.571429,161.607143,2966.0,2724.142857,2706.464286,42552.0,44568.428571,43738.035714
5213275,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,2016-06-18,11621,...,4083.678571,218.0,169.428571,161.107143,3266.0,2683.714286,2700.571429,51518.0,44169.857143,43829.321429


In [16]:
print(f"df memory usage: {np.round(df.memory_usage().sum()/(1024*1024),1)}mb")
df.to_pickle('./saved/data/'+'preprocessed.pkl')

df memory usage: 1302.6mb
